# Introduction to the Thomas Algorithm

In class, we showed how to calculate the solution to linear systems using Gaussian Elimination and LU Decomposition.

In the general case we studied we had $ A x = b $ where 
$$
    A = 
    \begin{pmatrix}
        A_{11} & A_{12} &  \cdots & A_{1N}  \\ 
        A_{21} & A_{22} &  \cdots & A_{2N}  \\ 
          \vdots & \vdots &        & \vdots \\ 
        A_{N1} & A_{N2} &  \cdots & A_{NN}  \\ 
    \end{pmatrix}
$$
***

#### Cost analysis of LU decomposition

In class we showed that a backsubstitution algorthm will be $\mathcal{O}(N^2)$ so all that is left is to see how expensive our Gaussian elimination will be. For simplicity, assume $+,-,\times,\div$ are all the same cost, 1 flop.

For each element in the first column $A_{21}$, $A_{31}$, ... , $A_{N1}$ we will need to do $N$ additions (one for each element of the given row). We will need to repeat this for each column, but each time, our starting point in the column and number of additions will each decrease by one.

$$
    \sum_{i=1}^{N} \sum_{j=i}^{N} \sum_{k=i}^{N+1} \mathcal{O}(1)
    \implies \sum_{i=1}^{N} (N - i)(N + 1 - i)
$$
If we let $j = N - i$, the sum simply reverses and we have 
$$
    \sum_{j=1}^{N} j(j+1) \implies \sum_{j=1}^{N} j^2 + \sum_{j=1}^{N} j
$$
These are a known sums, we end up with 
$$
   \frac{N^3}{3} + \frac{N^2}{2} + \frac{N}{6} + \frac{N(N+1)}{2} 
$$
Which is $\mathcal{O}(N^3)$, therefore Gaussian elimination dominates the time complexity.

**Summary: GE and LU decomposition are great for the general case, but quite slow.**

*Question: Are there are some special cases where we can do better?*

***

#### Tridiagonal systems
Noe that we have shown the cost of using LU decomposition to solve a problem, let's consider a problem where we should be able to do better. 

Consider the case of a initial value problem with $x \in (0, 1)$
$$
     \dot y = \frac{\partial^2 y}{\partial x^2}
$$
with initial conditions $y(x, 0) = g(x)$. 

*Note that I am using the notation* $\dot y = dy/dt$, $\ddot y = d^2y/dt^2$

Using finite differences, we can show that for our function $y(x, t)$
$$
    \frac{\partial^2 y}{\partial x^2} \approx \frac{y(x + h_x, t) - 2y(x, t) + f(x - h_x, t)}{h_x^2}
$$
With step size (in $x$) $h_x$ - this can be thought of as nodal spacing. *Please see (1) or (3) for explicit calculations of this.*

<!-- Include calculation in comments -->

What about in the time domain? Well we can compute a forward difference like so
$$
    \dot y \approx \frac{y(x, t + h_t) - y(x, t)}{h_t}
$$
So we get 
$$
    y(x, t + h_t) = y(x, t) + \frac{h_t}{h_x^2}\cdot[(y(x + h_x, t) - 2y(x, t) + f(x - h_x, t)]
$$
Now we have a way to calculate but we will run into issues if the following does not hold.
$$
    h_t \leq \frac{h_x^2}{2} 
$$
*See (3) or (4) for a proof of explicitly why this happens*

Despite not going into depth about why this happens, its maybe not so hard to see that there will be a problem coming from the fact that $\partial^2 y/\partial x^2$ is not calculated at $t + h_t$ (since we don't know the values that we would need to solve it).

We can introduce the Crank-Nicolson scheme and without going into too much detail this will mean that we average on the contribution to the second spatial derivative from timestep $t + h_t$.
$$
     -\frac{h_t}{2h_x^2}\cdot y(x + h_x, t + h_t) + (1 + \frac{h_t}{h_x^2})\cdot y(x, t + h_t) -\frac{h_t}{2h_x^2}\cdot y(x - h_x, t + h_t) \\= -\frac{h_t}{2h_x^2}\cdot y(x + h_x, t) + (1 + \frac{h_t}{h_x^2})\cdot y(x, t) -\frac{h_t}{2h_x^2}\cdot y(x - h_x, t)
$$
The point of all this is we must solve a tridiagonal linear system like the one below many times (at each timestep) if we want to be able to do better than the initial approach.

$$
    A = 
    \begin{pmatrix}
        A_{11} & A_{12} & 0         & 0         & \cdots     & 0     \\ 
        A_{12} & A_{22} & A_{23}    & 0         & \cdots     & 0     \\ 
        \vdots & \vdots & \vdots    & \vdots    & \vdots             \\ 
        0      & \cdots & 0         & 0         & A_{N,N-1}  & A_{NN}\\ 
    \end{pmatrix}
$$

For the Crank-Nicolson scheme, elements in each row will be, respectively,

$$
    -\frac{h_t}{2h_x^2}, (1 + \frac{h_t}{h_x^2}), -\frac{h_t}{2h_x^2}
$$

i.e. the same for the first and last elements.

<!-- The off diagonal elements are contributions from neighboring nodes. Note that it is common to use dummy/ghost points to enforce boundary conditions i.e. drop $A_{12}$ and $A_{N,N-1}$ terms. -->

But, if we try to use LU decomposition we will end up with $\mathcal{O}(N^3)$ for each time iteration, surely we can do better than this -- we have so many fewer eliminations to do!
***

Sources

1. E. Süli and D. Mayer, An Introduction to Numerical Analysis. Chapters 2 & 3
2. https://en.wikipedia.org/wiki/Tridiagonal_matrix_algorithm
3. A. Pierce, Math 316: Lecture 8 notes, retrieved from https://www.math.ubc.ca/~peirce/
4. https://www.quantstart.com/articles/Tridiagonal-Matrix-Solver-via-Thomas-Algorithm/
5. https://en.wikipedia.org/wiki/Crank–Nicolson_method